In [ ]:
# core imports
import os, time, json, glob
from pathlib import Path
from datetime import datetime
import pandas as pd
import numpy as np

import torch
from ultralytics import YOLO

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


In [ ]:
# ============================================
# Project Configuration
# ============================================

# ----- experiment configuration -----
RUN_TAG = datetime.now().strftime("%Y%m%d_%H%M%S")
PERSON_ONLY = True             # keep only 'person' class (COCO id=0)
IMG_SIZE     = 640
CONF_THRESH  = 0.25
MAX_IMAGES   = None            # set small int while testing (e.g., 100)

# ----- directory structure -----
ROOT = Path(__file__).resolve().parents[1] if "__file__" in globals() else Path.cwd().parents[0] 
os.chdir(ROOT)                          # current project root
DATA_DIR      = ROOT / "data"                        # raw dataset directory
DERIVED_DIR   = ROOT / "derived" / "caltech_yolo"    # derived local data
RUNS_DIR      = ROOT / "runs" / "phase2" / RUN_TAG   # experiment run directory
OUT_IMG_DIR   = RUNS_DIR / "annotated"               # annotated JPGs
OUT_CSV_DIR   = RUNS_DIR / "csv"                     # detection CSVs
FIG_DIR       = ROOT / "reports" / "figures" / RUN_TAG  # plots and visualizations

# make sure all exist
for p in [DATA_DIR, DERIVED_DIR, RUNS_DIR, OUT_IMG_DIR, OUT_CSV_DIR, FIG_DIR]:
    p.mkdir(parents=True, exist_ok=True)

# ----- dataset source -----
# Option A: use existing local dataset (preferred for speed)
# Option B: use Roboflow if you want an on-demand cloud dataset
USE_ROBOFLOW = True

if not USE_ROBOFLOW:
    # Use derived/caltech_yolo images folder
    DATA_IMG_DIR = DERIVED_DIR / "images" / "test"
else:
    # Roboflow download (expects env var ROBOFLOW_API_KEY)
    from roboflow import Roboflow

    api_key = os.environ.get("ROBOFLOW_API_KEY")
    assert api_key, "Set ROBOFLOW_API_KEY in environment to download dataset securely."

    rf = Roboflow(api_key=api_key)
    project  = rf.workspace("raditya").project("caltech-pedestrian-iitp")
    version  = project.version(1)
    ds       = version.download("yolov11")   # YOLOv11 format images

    # try to find a valid split
    candidates = [
        Path(ds.location) / "valid" / "images",
        Path(ds.location) / "test" / "images",
        Path(ds.location) / "train" / "images",
        Path(ds.location) / "images",   # fallback
    ]
    DATA_IMG_DIR = next((c for c in candidates if c.exists()), None)
    assert DATA_IMG_DIR and any(DATA_IMG_DIR.glob("*.jpg")), f"No images found under {ds.location}"

print(f"\n Directory Structure:")
print("ROOT:", ROOT)
print("DATA_DIR:", DATA_DIR)
print("DERIVED_DIR:", DERIVED_DIR)
print("RUNS_DIR:", RUNS_DIR)
print("FIG_DIR:", FIG_DIR)
print("Images dir:", DATA_IMG_DIR)


In [ ]:
# ============================================
# Load Pretrained Models from Ultralytics Hub
# (no local .pt weights needed)
# ============================================

print("Loading models from Ultralytics Hub...")

# Load directly from Ultralytics registry
yolo_model   = YOLO("yolov11m.pt")    # baseline 
rtdetr_model = YOLO("rtdetr-l.pt")   # RT-DETR Large (Ultralytics implementation)

print("Models loaded successfully.")
print(f"YOLO device: {yolo_model.device}")
print(f"RT-DETR device: {rtdetr_model.device}")

In [ ]:
# ============================================
# Run Inference on Dataset (YOLOv8n & RT-DETR-L)
# ============================================

from tqdm import tqdm

# quick sanity check
assert DATA_IMG_DIR.exists(), f"Dataset path not found: {DATA_IMG_DIR}"
image_files = sorted([p for p in DATA_IMG_DIR.glob("*.jpg")])
if MAX_IMAGES:
    image_files = image_files[:MAX_IMAGES]

print(f"🖼️ Found {len(image_files)} images for inference in {DATA_IMG_DIR}")

# unified inference settings
predict_kwargs = dict(
    imgsz=IMG_SIZE,
    conf=CONF_THRESH,
    save=True,
    save_txt=True,
    save_conf=True,
    project=str(OUT_IMG_DIR),
    name="predictions",
    exist_ok=True,
    verbose=False
)
if PERSON_ONLY:
    predict_kwargs["classes"] = [0]  # COCO 'person'

# helper function for single model inference
def run_inference(model, model_name):
    """Run inference on all images and return a detections DataFrame."""
    rows = []
    print(f"\n🚀 Running inference with {model_name}...")

    for img_path in tqdm(image_files, desc=model_name):
        results = model(img_path, **predict_kwargs, stream=False)
        res = results[0]

        h, w = res.orig_shape
        inf_ms = float(res.speed.get("inference", 0.0)) if hasattr(res, "speed") else 0.0

        for b in res.boxes:
            cls_id = int(b.cls)
            conf = float(b.conf)
            x1, y1, x2, y2 = map(float, b.xyxy[0].tolist())

            rows.append({
                "image_path": str(img_path),
                "model": model_name,
                "class_id": cls_id,
                "class_name": model.names.get(cls_id, str(cls_id)),
                "conf": conf,
                "x1": x1, "y1": y1, "x2": x2, "y2": y2,
                "width": w, "height": h,
                "inf_ms": inf_ms
            })

    df = pd.DataFrame(rows)
    print(f"✅ {model_name}: {len(df)} detections saved.")
    return df


# ---- Run both models ----
df_yolo   = run_inference(yolo_model, "YOLOv8n")
df_rtdetr = run_inference(rtdetr_model, "RT-DETR-L")

# ---- Merge and save ----
df_all = pd.concat([df_yolo, df_rtdetr], ignore_index=True)

out_csv = OUT_CSV_DIR / "phase2_detections.csv"
df_all.to_csv(out_csv, index=False)

print(f"\n📦 All detections saved to: {out_csv.resolve()}")
display(df_all.head())


In [ ]:
# ============================================
# Spark: Load and Aggregate Detection Results
# ============================================

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# ---- Initialize Spark ----
spark = SparkSession.builder \
    .appName("PedestrianDetectionPhase2") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

print("Spark Session Started")

# ---- Load YOLO + RT-DETR detections ----
csv_path = str(OUT_CSV_DIR / "phase2_detections.csv")
df_spark = spark.read.csv(csv_path, header=True, inferSchema=True)
print(f"📂 Loaded detections from {csv_path}")

# ---- Aggregate Metrics ----
summary = (
    df_spark.groupBy("model")
    .agg(
        F.count("*").alias("num_detections"),
        F.mean("conf").alias("mean_confidence"),
        F.stddev("conf").alias("std_confidence"),
        F.mean("inf_ms").alias("mean_inference_ms"),
        F.min("inf_ms").alias("min_inference_ms"),
        F.max("inf_ms").alias("max_inference_ms"),
        F.countDistinct("image_path").alias("num_images")
    )
    .orderBy("model")
)

summary_pd = summary.toPandas()
display(summary_pd)

# ---- Save Spark Summary ----
summary_csv = OUT_CSV_DIR / "phase2_summary.csv"
summary_pd.to_csv(summary_csv, index=False)
print(f"Summary saved to {summary_csv.resolve()}")


In [ ]:
# ============================================
# Visualization: YOLOv11m vs RT-DETR-L
# ============================================

import matplotlib.pyplot as plt
import seaborn as sns

# ---- Load Spark summary output ----
summary_csv = OUT_CSV_DIR / "phase2_summary.csv"
df_summary = pd.read_csv(summary_csv)

# ---- Visualization output dir ----
FIG_DIR.mkdir(parents=True, exist_ok=True)
print("Figures will be saved to:", FIG_DIR.resolve())

sns.set(style="whitegrid", context="talk", palette="tab10")

# ---------- 1️⃣ Confidence Distribution ----------
plt.figure(figsize=(8,5))
sns.barplot(data=df_summary, x="model", y="mean_confidence", hue="model", dodge=False)
plt.title("Mean Detection Confidence by Model")
plt.ylabel("Mean Confidence")
plt.xlabel("")
plt.tight_layout()
plt.savefig(FIG_DIR / "confidence_comparison.png", dpi=200)
plt.close()

# ---------- 2️⃣ Inference Latency ----------
plt.figure(figsize=(8,5))
sns.barplot(data=df_summary, x="model", y="mean_inference_ms", hue="model", dodge=False)
plt.title("Mean Inference Latency (ms) per Model")
plt.ylabel("Inference Time (ms)")
plt.xlabel("")
plt.tight_layout()
plt.savefig(FIG_DIR / "latency_comparison.png", dpi=200)
plt.close()

# ---------- 3️⃣ Detections per Image ----------
plt.figure(figsize=(8,5))
sns.barplot(data=df_summary, x="model", y="num_detections", hue="model", dodge=False)
plt.title("Total Detections per Model")
plt.ylabel("Number of Detections")
plt.xlabel("")
plt.tight_layout()
plt.savefig(FIG_DIR / "detections_comparison.png", dpi=200)
plt.close()

# ---------- 4️⃣ Combined Summary Plot ----------
fig, ax1 = plt.subplots(figsize=(8,5))
sns.scatterplot(
    data=df_summary,
    x="mean_confidence",
    y="mean_inference_ms",
    size="num_detections",
    hue="model",
    ax=ax1,
    sizes=(100, 500)
)
plt.title("Confidence vs. Latency Trade-off")
plt.xlabel("Mean Confidence")
plt.ylabel("Mean Inference Time (ms)")
plt.tight_layout()
plt.savefig(FIG_DIR / "confidence_vs_latency.png", dpi=200)
plt.close()

print("Comparison visualizations generated successfully.")
print(f"All figures saved to: {FIG_DIR.resolve()}")


In [ ]:
# ============================================
# Phase 2 - Report Packaging
# ============================================

import shutil
from datetime import datetime

SUMMARY_DIR = ROOT / "reports" / "phase2" / "summary"
SUMMARY_DIR.mkdir(parents=True, exist_ok=True)

# ---- copy top visualizations ----
key_figs = [
    FIG_DIR / "confidence_comparison.png",
    FIG_DIR / "latency_comparison.png",
    FIG_DIR / "detections_comparison.png",
    FIG_DIR / "confidence_vs_latency.png",
]

for f in key_figs:
    if f.exists():
        shutil.copy2(f, SUMMARY_DIR / f.name)

# ---- generate markdown summary ----
markdown_summary = f"""# 🚦 Phase 2 Model Benchmark Summary
**Date:** {datetime.now().strftime("%Y-%m-%d %H:%M")}  
**Models Compared:** YOLOv11m vs RT-DETR-L  
**Dataset:** Caltech Pedestrian (IITP via Roboflow)

---

## 📊 Key Metrics
| Model | Mean Confidence | Mean Latency (ms) | Detections |
|:------|----------------:|------------------:|------------:|
"""

for _, row in df_summary.iterrows():
    markdown_summary += f"| {row['model']} | {row['mean_confidence']:.3f} | {row['mean_inference_ms']:.2f} | {row['num_detections']} |\n"

markdown_summary += f"""

---

## 🖼️ Visual Comparisons
![Confidence Distribution](confidence_comparison.png)  
![Latency Histogram](latency_comparison.png)  
![Detections Comparison](detections_comparison.png)  
![Confidence vs Latency](confidence_vs_latency.png)

---

### 🧾 Summary Notes
- **YOLOv11m** demonstrates balanced speed and accuracy, achieving competitive confidence scores with lower latency than transformer-based RT-DETR-L.  
- **RT-DETR-L** excels in detection recall and robustness under complex backgrounds, though at a higher inference cost.  
- Ideal trade-off model depends on deployment: real-time → YOLOv11m; high-fidelity offline → RT-DETR-L.  

✅ Figures and summary saved to: `{SUMMARY_DIR.resolve()}`
"""

# ---- save markdown file ----
md_path = SUMMARY_DIR / "phase2_summary.md"
with open(md_path, "w") as f:
    f.write(markdown_summary)

print("✅ Phase 2 summary report exported successfully.")
print(f"📄 Markdown summary: {md_path.resolve()}")
print(f"📁 Figures copied to: {SUMMARY_DIR.resolve()}")
